# Oject detection evaluation with SAHI

Test a Detector on A Customized Dataset using Slicing Aided Hyper Inference (SAHI). Here, I use the Faster RCNN

https://github.com/obss/sahi

https://medium.com/voxel51/how-to-detect-small-objects-cfa569b4d5bd


- Import required modules:

In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer, ColorMode
import matplotlib.pyplot as plt
import cv2
import os
from detectron2.data.datasets import register_coco_instances

from sahi.utils.detectron2 import Detectron2TestConstants

# import required functions, classes
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction, predict, get_prediction
from sahi.utils.file import download_from_url
from sahi.utils.cv import read_image
from IPython.display import Image

## Load the custom model

In [ ]:
model_checkpoint_path = "/scratch/tjian/PythonProject/deep_plastic_Flux_SSL/checkpoint/train_weights/RN50_500K/Exp1/RN50_500K_200e/SSL_100per_1/model_best_83.6285.pth"
model_config_path = "/scratch/tjian/PythonProject/deep_plastic_Flux_SSL/checkpoint/train_weights/RN50_500K/Exp1/RN50_500K_200e/SSL_100per_1/config.yaml"

## Register my custom train and test dataset

dataset should be in COCO format

In [ ]:
# Register a test dataset
register_coco_instances("Flux_train_100per", {}, "/scratch/tjian/Data/Flux/labeled_images_new/annotations/Train_100per.json", "/scratch/tjian/Data/Flux/labeled_images_new/Train_100per/")
register_coco_instances("Flux_val_100per", {}, "/scratch/tjian/Data/Flux/labeled_images_new/annotations/Val_100per.json", "/scratch/tjian/Data/Flux/labeled_images_new/Val_100per/")
Train_Dataset_name="Flux_train_100per"
Test_Dataset_name="Flux_val_100per"
Class_name=["litter"]

MetadataCatalog.get(Train_Dataset_name).set(thing_classes=Class_name)
MetadataCatalog.get(Test_Dataset_name).set(thing_classes=Class_name)

## Build a detection model

Instantiate a detection model by defining model weight path, confing path and other parameters.

This model will be used to inference in slicing images or original images.

In [ ]:
# "image_size": Input image size for each inference (image is scaled by preserving aspect ratio).
# there is no default "image_size". 
# So if not set "image_size", the image will be resized by the setting in the model config file

# "device": (1) "cpu", or (2) 'cuda:0'

detection_model = AutoDetectionModel.from_pretrained(
    model_type='detectron2',
    model_path=model_checkpoint_path,
    config_path=model_config_path,
    confidence_threshold=0.9,
    # image_size=1333,
    device="cuda:0",
)

## Sliced Inference

### 1. Predict an image

Sliced Inference with a Detectron2 Model

- To perform sliced prediction we need to specify slice parameters. In this example we will perform prediction over slices of slice_width*slice_height (e.g.,256x256) with an overlap ratio of 0.2:

In [ ]:
# define image path
image_path=r"/scratch/tjian/Data/Flux/TUD_Vietnam/0609/images_with_litter/BL_2023_09_06_Sweep_1_5_DCSS0262.jpg"

export_dir=r"/scratch/tjian/Data/Flux/TUD_Vietnam/0609/Predict/R50_300K_100e/SAHI/test/"
output_file_name= "BL_2023_09_06_Sweep_1_5_DCSS0262"   # do not end with ".jpg"

result = get_sliced_prediction(
    image_path,
    detection_model,
    slice_height = 640,
    slice_width = 640,
    overlap_height_ratio = 0.2,
    overlap_width_ratio = 0.2,
)

result.export_visuals(export_dir, 
                      text_size=3,
                      file_name=output_file_name)

# output No. detections
num_sliced_dets = len(result.to_fiftyone_detections())
print(f"Detections predicted with slicing: {num_sliced_dets}")

Performing prediction on 15 number of slices.


### 2. Predict images in a folder

Output:

(1) predicted images (the code will save the predicted images in a subfolder named *"pred_images"*); 

(2) A excel file named *"detection_results.xlsx"*, stores filename and No.detections

(3) Print the total number of detections in the console

"postprocess_type": Options are 'NMM', 'GREEDYNMM', 'LSNMS' or 'NMS'. Default is 'GREEDYNMM'.

In [ ]:
source_image_dir = r"/scratch/tjian/Data/Flux/TUD_Vietnam/1209/images_no_litter/"

# do not generate the output_dir, the code will do it automatically
output_dir = r"/scratch/tjian/Data/Flux/TUD_Vietnam/1209/Pred/SSL_20per_7/SA_1920_Conf_0.9/No_litter"

# define slicing conf.
slice_height = 1920
slice_width = 1920
overlap_height_ratio = 0.2
overlap_width_ratio = 0.2

# Perform sliced inference on given folder
# it will also output the (total) number of detections in each/all images
result = predict(
    detection_model = detection_model,
    source = source_image_dir,
    slice_height = slice_height,
    slice_width = slice_width,
    overlap_height_ratio = overlap_height_ratio,
    overlap_width_ratio = overlap_width_ratio,
    project = output_dir,
    postprocess_type = 'NMS', # dDefault is 'GREEDYNMM'.
    postprocess_match_metric = 'IOU',
    postprocess_match_threshold = 0.5, # NMS_IoU_threshold
    visual_text_size = 3 # float
)

### 3. Evaluate models on a labeled dataset

In the output path, the code generates:　

(1) predicted images (the code will save the predicted images in a subfolder named *"pred_images"*); 

(2) a subfolder named *"visuals_with_gt"* including images with ground-truth and predictions;

(3) a json file "results.json" including prediction results;

(4) A excel file named *"detection_results.xlsx"*, stores filename and No.detections

(5) Print the total number of detections in the console

#### 3.1 Processing images

In [ ]:
test_image_dir = r"/scratch/tjian/Data/Flux/TUD_Vietnam/1209/Sweep_4_5/images/"

# test dataset (with annotations)
test_annotation_path = r"/scratch/tjian/Data/Flux/TUD_Vietnam/1209/Sweep_4_5/Sweep_4_5.json"

# do not generate the output_dir, the code will do it automatically
output_dir = r"/scratch/tjian/Data/Flux/TUD_Vietnam/1209/Sweep_4_5/Pred/"

# define slicing conf.
slice_height = 1920
slice_width = 1920
overlap_height_ratio = 0.2
overlap_width_ratio = 0.2

# Perform sliced inference on given folder:
# it will also output the (total) number of detections in each/all images
predict(
    detection_model = detection_model,
    source = test_image_dir,
    slice_height = slice_height,
    slice_width = slice_width,
    overlap_height_ratio = overlap_height_ratio,
    overlap_width_ratio = overlap_width_ratio,
    project = output_dir,
    dataset_json_path = test_annotation_path,
    postprocess_type = 'NMS', # dDefault is 'GREEDYNMM'.
    postprocess_match_metric = 'IOU',
    postprocess_match_threshold = 0.5, # NMS_IoU_threshold
    visual_text_size = 3 # float
    # visual_bbox_thickness = None  # init
)

#### 3.2 Modify the output json file

Note: the "results.json" file output in the above code (1) has an issue of category. 

By default, Detectron2 or SAHI starts the category_id from "0". But the category_id starts from "1" in my input annotation file. 

Thus, it is **necessary** that all category IDs are incremented by 1 in the output json file.

In [ ]:
import json

input_json = r'/scratch/tjian/Data/Flux/TUD_Vietnam/1209/Sweep_4_5/Pred/results.json'
output_json = r'/scratch/tjian/Data/Flux/TUD_Vietnam/1209/Sweep_4_5/Pred/results_new.json'

# Read the JSON file
with open(input_json, 'r') as file:
    coco_data = json.load(file)

# Check if the JSON structure is a list or a dictionary
if isinstance(coco_data, list):
    # Assume the list contains annotation dictionaries directly
    for annotation in coco_data:
        annotation['category_id'] += 1
else:
    # Assume it's a dictionary with an 'annotations' key
    for annotation in coco_data.get('annotations', []):
        annotation['category_id'] += 1

# Save the modified JSON to a new file
with open(output_json, 'w') as file:
    json.dump(coco_data, file, indent=4)

print(f"All category IDs have been incremented by 1 and saved to {output_json}")


#### 3.3 Output evaulation metrics in "main_Confusion_matrix_OD.ipynb"

e.g., mAP, TP, FP, FN, precision, recall, F1-score

- Predictions are returned as [sahi.prediction.PredictionResult](sahi/prediction.py), you can access the object prediction list as: